In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
data = pd.read_csv('movies_metadata.csv')
keywords = pd.read_csv('keywords.csv')
posterContext = pd.read_excel('finalOutput.xlsx')


FileNotFoundError: ignored

In [1]:
import os
import json
import numpy as np
from numpy import nan
import requests
import base64
import time
import requests

errors_dict = {}
posters_imdbId_context_dict = {}
for index, row in data.iterrows():
    if index>10000:
      break
    id = row['id']
    """if type(path) != str:
        continue
    url = "https://image.tmdb.org/t/p/w500" + path"""

    # Replace the API_KEY with your TMDb API key
    API_KEY = "270270948898f73f4cc3bd80e8778c3d"
    # Replace the movie_id with the TMDb ID of the movie you want to retrieve the poster for
    movie_id = id

    # Make the request to the TMDb API to retrieve the movie details
    response = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={API_KEY}&language=en-US")

    # Check if the request was successful
    if response.status_code != 200:
        print("Error retrieving movie details")
        exit()

    # Retrieve the poster path from the movie details
    poster_path = response.json().get("poster_path")

    # Check if the movie has a poster
    if not poster_path:
        print("Movie does not have a poster")
        exit()

    # Make the request to the TMDb API to retrieve the poster image
    response = requests.get(f"https://image.tmdb.org/t/p/original{poster_path}")

    # Check if the request was successful
    if response.status_code != 200:
        print("Error retrieving poster image")
        continue

    img_data = response.content
    os.makedirs("posters", exist_ok=True)
    img_location = 'posters/'+str(id)+'_poster.jpg'
    with open(img_location, 'wb') as handler:
        handler.write(img_data)
    with open(img_location, "rb") as img_file:
        my_string = base64.b64encode(img_file.read())
#     print(my_string.decode("utf-8"))
    hugging_face_url = "https://snehithb-get-image-context.hf.space/api/queue/push/"
    hugging_face_status_url = "https://snehithb-get-image-context.hf.space/api/queue/status/"
    
#     Needs to be removed, added to test for 1 case
#     if index > 1:
#         continue
    my_string = "data:image/jpeg;base64,"+my_string.decode("utf-8")
    payload = json.dumps({
      "fn_index": 0,
      "data": [
      my_string
    ],
      "action": "predict",
      "session_hash": "7p1oh5oyfh8"
    })
    headers = {
      'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/111.0',
      'Accept': '*/*',
      'Accept-Language': 'en-US,en;q=0.5',
      'Accept-Encoding': 'gzip, deflate, br',
      'Referer': 'https://srddev-image-caption.hf.space/?__theme=light',
      'Content-Type': 'application/json',
      'Origin': 'https://srddev-image-caption.hf.space',
      'Connection': 'keep-alive',
      'Cookie': 'session-space-cookie=acb1440f515e0f84f41ddb669ad094d1',
      'Sec-Fetch-Dest': 'empty',
      'Sec-Fetch-Mode': 'cors',
      'Sec-Fetch-Site': 'same-origin',
      'TE': 'trailers'
    }
    try:
        response = requests.request("POST", hugging_face_url, headers=headers, data=payload)
    #     print(response.text["hash"])
    #     payload = json.dumps({
    #       response.text
    #     })
        temp = {}
        flag = 0
        while True:
            flag = flag + 1
            time.sleep(5) # average duration for hugging face to get the context of poster
            response1 = requests.request("POST", hugging_face_status_url, headers=headers, data=response)
            if flag == 5:
              break
#             print(response1.text)
            temp = json.loads(response1.text)
        #     print(temp)
            if temp['status'] == 'COMPLETE':
               break
#         print(temp['data']['data'][0])
        posters_imdbId_context_dict[id] = temp['data']['data'][0]
    except Exception as e:
        errors_dict[index] = e
    print(posters_imdbId_context_dict)

NameError: ignored

In [ ]:
posters_imdbId_context_dict

{}